<a href="https://colab.research.google.com/github/Vizzuality/copernicus-climate-data/blob/master/cds_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data for the copernicus climate project

https://github.com/Vizzuality/copernicus-climate-data

`Edward P. Morris (vizzuality.)`

### Description
This code accesses data from the Copernicus Climate Data Service, using as much as possible the cds toolbox to preprocess the data files. NetCDF4 files are created and saved to cloud storage. Datasets are combined and exported to cloud storage in the cloud optimised ZARR format, ready for further analysis.

### Notes
+ The cds toolbox is in development, some parts of this script maybe available in the toolbox in the future.

+ Requests to the cds toolbox require authorisation credentials, and one must agree to the license of each individual dataset! see https://confluence.ecmwf.int/display/CKB/Climate+Data+Store+%28CDS%29+documentation

+ Requests to the cds toolbox may take a very long time, once a request has been processed it is available as a cached result, hence upon the first run of this script it is useful to run cells with cds requests, cancel them, and rerun the script after the result is ready (check this at https://cds.climate.copernicus.eu/cdsapp#!/yourrequests)

### TODO

+ add variables needed for PET calculations
+ check if seasonal and future predictions require correction for bias
+ consider correcting ERA5-Land bias for specific region (using insitu data)
+ check if conversion to CF dates is always correct
+ check why ERA5-Land gives error about continuous dates
+ consider adding more realizations and models to the CMIP5 dataset
+ investigate the optimum chunk configeration for each dataset
+ tidy up each datsets metadata
    

```
MIT License

Copyright (c) 2020 Vizzuality

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

```


# Setup

## Linux dependencies

In [0]:
!apt install libspatialindex-dev

## Python packages

In [0]:
!pip install cftime netcdf4 zarr xarray xclim cdsapi cdstoolbox-remote gcsfs

## Authorisation

In [0]:
# For auth WITHOUT service account
#from google.colab import auth
#auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#project_id = "skydipper-196010"
#!gcloud config set project {project_id}

In [0]:
# Copy GC credentials to home (place in your GDrive, and connect Drive)
!cp "/content/drive/My Drive/skydipper-196010-f842645fd0f3.json" "/root/.skydipper-196010-f842645fd0f3.json"

In [5]:
# Auth WITH service account
!gcloud auth activate-service-account \
  edward-morris@skydipper-196010.iam.gserviceaccount.com \
          --key-file=/root/.skydipper-196010-f842645fd0f3.json --project="skydipper-196010"


Activated service account credentials for: [edward-morris@skydipper-196010.iam.gserviceaccount.com]


In [6]:
# Test GC auth
!gsutil ls "gs://copernicus-climate"

gs://copernicus-climate/RH_ERA5.zip
gs://copernicus-climate/mrt_ERA5-HEAT.zip
gs://copernicus-climate/sp_ERA5.zip
gs://copernicus-climate/t2m_ERA5.zip
gs://copernicus-climate/variables_futuro_CMIP5_AEMET.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_1.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_2.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_3.zip
gs://copernicus-climate/variables_futuro_CMIP5_ERA5land_4.zip
gs://copernicus-climate/variables_futuro_seasonal_ERA5land_1.zip
gs://copernicus-climate/variables_futuro_seasonal_ERA5land_2.zip
gs://copernicus-climate/variables_futuro_seasonal_ERA5land_3.zip
gs://copernicus-climate/ws_ERA5.zip
gs://copernicus-climate/dataset/
gs://copernicus-climate/spain.zarr/


In [0]:
# Copy CDS API credentials to home (place in your GDrive, and connect Drive)
!cp "/content/drive/My Drive/cdsapirc" "/root/.cdsapirc"


In [8]:
# Clean up
!rm -r sample_data

rm: cannot remove 'sample_data': No such file or directory


# Utils

## mkdirs

In [0]:
from pathlib import Path

def mkdirs(dirs_list, exist_ok=True):
  """ Create nested directories
  """
  for p in dirs_list:
    Path(p).mkdir(parents=True, exist_ok=exist_ok)


## copy_gcs

In [0]:
import os
import subprocess

def copy_gcs(source_list, dest_list, opts=""):
  """
  Use gsutil to copy each corresponding item in source_list
  to dest_list

  """
  for s, d  in zip(source_list, dest_list):
    cmd = f"gsutil -m cp -r {opts} {s} {d}"
    print(f"Processing: {cmd}")
    r = subprocess.call(cmd, shell=True)
    if r == 0:
        print("Task created")
    else:
        print("Task failed")
  print("Finished copy")

## list_paths

In [0]:
import glob
import subprocess

def list_paths(uri_prefix, dir_path, file_pattern="*", gsutil=True, return_dir_path=True):
        ''' Creates a list of full paths 
    
        Uses glob regex rules allowing flexible patterns
    
        Parameters
        ----------
        uri_prefix : str
            The (GCS) uri prefix.
        dir_path : str
            Directory path, can use regex.
        file_pattern : str
            File pattern for glob searching.
        gsutil : bool
            Use gsutil, default is True.
        return_dir_path : bool
            Return directory path relative to uri_prefix, default is True.        
    
        Returns
        -------
        List of path strings.
        
        Examples
        --------
        # Requires authentication
        #list_paths("gs://skydipper-water-quality", "cloud-masks/*", "*.tif", True, False)
        '''
        p = f"{uri_prefix}/{dir_path}/{file_pattern}"
        print(f"Searching {p}")
        if not gsutil:
          out = glob.glob(p)
        if gsutil:
          cmd = f"gsutil ls {p}"
          out = subprocess.check_output(cmd, shell=True).decode('utf8').split('\n')
          out.pop(-1)
        if return_dir_path:
          out = [f.split(uri_prefix)[1] for f in out]  
        print(f"Found {len(out)} path(s)")
        return out

#path_list = list_paths("gs://skydipper-water-quality", "cloud-masks/*", "*.tif", True, True)
#print(path_list[0])



## extract_string

In [0]:
import os
def extract_string(file_path, split, index):
  """
  Get string by splitting path
  
  @arg file_path The file path string to split
  @arg split Caracter to split path on
  @index Index integer to retain

  @return A string
  """ 
  return os.path.splitext(file_path)[0].split(split)[index]

## set_acl_to_public

In [0]:
import subprocess

# Set to asset permissions to public for https read
def set_acl_to_public(gs_path):
  """ 
  Set all Google Storage assets to puplic read access.

  Requires GS authentication

  Parameters
  ----------
  gs_path str
    The google storage path, note the "-r" option is used, setting the acl of all assets below this path
  """
  cmd = f"gsutil -m acl -r ch -u AllUsers:R {gs_path}"
  print(cmd)
  r = subprocess.call(cmd, shell=True)
  if r is 0:
    print("Set acl(s) sucsessful")
  else:
    print("Set acl(s) failed")  

#set_acl_to_public("gs://skydipper-water-quality/cloud-masks")

## unchunk_dataset

In [0]:
# unchunk coords
# from xcube
import json
import os.path
from typing import List, Sequence

import numpy as np
import xarray as xr
import zarr


def unchunk_dataset(dataset_path: str, var_names: Sequence[str] = None, coords_only: bool = False):
    """
    Unchunk dataset variables in-place.
    :param dataset_path: Path to ZARR dataset directory.
    :param var_names: Optional list of variable names.
    :param coords_only: Un-chunk coordinate variables only.
    """

    is_zarr = os.path.isfile(os.path.join(dataset_path, '.zgroup'))
    if not is_zarr:
        raise ValueError(f'{dataset_path!r} is not a valid Zarr directory')

    with xr.open_zarr(dataset_path) as dataset:
        if var_names is None:
            if coords_only:
                var_names = list(dataset.coords)
            else:
                var_names = list(dataset.variables)
        else:
            for var_name in var_names:
                if coords_only:
                    if var_name not in dataset.coords:
                        raise ValueError(f'variable {var_name!r} is not a coordinate variable in {dataset_path!r}')
                else:
                    if var_name not in dataset.variables:
                        raise ValueError(f'variable {var_name!r} is not a variable in {dataset_path!r}')

    _unchunk_vars(dataset_path, var_names)


def _unchunk_vars(dataset_path: str, var_names: List[str]):
    for var_name in var_names:
        var_path = os.path.join(dataset_path, var_name)

        # Optimization: if "shape" and "chunks" are equal in ${var}/.zarray, we are done
        var_array_info_path = os.path.join(var_path, '.zarray')
        with open(var_array_info_path, 'r') as fp:
            var_array_info = json.load(fp)
            if var_array_info.get('shape') == var_array_info.get('chunks'):
                continue

        # Open array and remove chunks from the data
        var_array = zarr.convenience.open_array(var_path, 'r+')
        if var_array.shape != var_array.chunks:
            # TODO (forman): Fully loading data is inefficient and dangerous for large arrays.
            #                Instead save unchunked to temp and replace existing chunked array dir with temp.
            # Fully load data and attrs so we no longer depend on files
            data = np.array(var_array)
            attributes = var_array.attrs.asdict()
            # Save array data
            zarr.convenience.save_array(var_path, data, chunks=False, fill_value=var_array.fill_value)
            # zarr.convenience.save_array() does not seem save user attributes (file ".zattrs" not written),
            # therefore we must modify attrs explicitly:
            var_array = zarr.convenience.open_array(var_path, 'r+')
            var_array.attrs.update(attributes)

## get_cached_zarr

In [0]:
import gcsfs
import zarr
import xarray as xr

def get_cached_zarr(
    group,
    project_id = "skydipper-196010",
    root = "copernicus-climate/spain.zarr",
    token="/root/.skydipper-196010-f842645fd0f3.json"
    ):
  
  # Connect to GS
  gc = gcsfs.GCSFileSystem(project=project_id, token=token)
  path_or_store = gcsfs.mapping.GCSMap(root, gcs=gc, check=True, create=False)
  # Check zarr is consolidated
  consolidated = gc.exists(f'{root}/.zmetadata')
  # Cache the zarr store
  cache = zarr.LRUStoreCache(path_or_store, max_size=2**28)
  # Return cached zarr group
  return xr.open_zarr(cache, group=group, consolidated=consolidated)

# Data packages

Data from the Copernicus Climate Data Service ([CDS](https://cds.climate.copernicus.eu)).

## Region of interest 
Spain (mainland) using bounding box `[44.0, -10.0, 35.0, 5.0] #NWSE` 

## Historical:
[ERA5-Land hourly data from 1981 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview)

+ temporal_resolution: `hour` (convert to daily using cdstoolbox)
+ variables: `'2m_temperature'` (convert to daily min and max using cdstoolbox)
+ grid: `[0.1, 0.1]`
+ time_interval: `1981--2020`, all months, days, and hours

[Thermal-comfort-indices-derived-from-ERA5-reanalysis](https://cds.climate.copernicus.eu/cdsapp#!/dataset/derived-utci-historical?tab=overview)

+ temporal_resolution: `day`
+ variables: "Mean radiant temperature", 'Universal thermal climate index'
+ grid: `[0.25, 0.25]`
+ time_interval: `1979--2020`, all months, and days

## Future seasonal

[Seasonal forecast daily data on single levels from 2017 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-original-single-levels?tab=doc)

+ temporal_resolution: `day`
+ variables: `'maximum_2m_temperature_in_the_last_24_hours', 'minimum_2m_temperature_in_the_last_24_hours'`
+ originating_centre: `'cmcc', 'dwd', 'ecmwf', 'meteo_france', 'ncep', 'ukmo'` 
+ grid: depends on model, interpolate to `[0.1, 0.1]`
+ time_interval: `2019-10-01--2020-04-01`
+ realizations: `[50]`, varies between models 

## Future long-term

[CMIP5 daily data on single levels](https://cds.climate.copernicus.eu/cdsapp#!/dataset/projections-cmip5-daily-single-levels?tab=overview)

+ temporal_resolution: `day`
+ variables: `'minimum_2m_temperature_in_the_last_24_hours', 'maximum_2m_temperature_in_the_last_24_hours'`
+ experiments: `'rcp_4_5', 'rcp_8_5'`
+ models: `'bnu_esm', 'access1_0', 'mpi_esm_lr', 'mpi_esm_mr', 'noresm1_m'`
+ realizations: `'r1i1p1'`
+ grid: depends on model, interpolate to `[0.1, 0.1]`
+ time_interval: `2006--2100`




### Setup directory structure

In [0]:
# Define paths
ds_dir = "dataset"
gs_prefix ="gs://copernicus-climate"
zarr_dir = "spain.zarr"
dirs_list = [f"{ds_dir}/seasonal-original-single-levels", f"{ds_dir}/projections-cmip5-daily-single-levels", f"{ds_dir}/reanalysis-era5-land"]            

In [17]:
# Start with a clean structure
!rm -r {ds_dir}
!rm -r {zarr_dir}
!rm *.nc

rm: cannot remove '*.nc': No such file or directory


In [0]:
# Make directory structure
mkdirs(dirs_list, exist_ok=True)

In [0]:
# Start with clean GCS ZARR
#!gsutil -m rm -r gs://copernicus-climate/spain.zarr

In [0]:
# Make zarr dir
import zarr
root = zarr.open(zarr_dir, mode='a') 

## Seasonal forecast daily data on single levels from 2017 to present

### Get data from CDS toolbox

In [21]:
%%writefile download.py
import cdstoolbox as ct
import datetime

# Out directory
ds_dir = "dataset"

# Define region of interest
bbox = [35, -10, 44, 5]
#grid = [0.1, 0.1]

# Define datasets to download
var_list = ['minimum_2m_temperature_in_the_last_24_hours', 'maximum_2m_temperature_in_the_last_24_hours']
center_list = ['cmcc', 'dwd', 'ecmwf', 'meteo_france', 'ncep', 'ukmo']
leadtime_list = [str(i) for i in range(24, 5184, 24)] 
system_list = ['3', '2', '5', '7', '2', '14']
day = '01'
month = '10'
year = '2019'

# Function to create properties dict for download
def create_data_dict(variable, center, system, leadtime_list, day, month, year):
  return {
        'originating_centre': center,
        'system': system,
        'variable': variable,
        'year': year,
        'month': month,
        'day': day,
        'leadtime_hour': leadtime_list,
    }

# Make list of properties dicts, and out names
pdict_list = []
nm_list = []
for center, system in zip(center_list, system_list):
    pdict_list.append(create_data_dict(var_list, center, system, leadtime_list, day, month, year))
        
    # make file names
    sd = datetime.datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d")
    ed = sd + datetime.timedelta(days=int(leadtime_list[-1])/24)
    ed = ed.strftime('%Y-%m-%d')
    ti = f"{year}-{month}-{day}--{ed}"
    v = 'tasmin-tasmax' #variable.replace("_", "-")
    nm_list.append(f"bbox-{bbox[0]}-{bbox[1]}-{bbox[2]}-{bbox[3]}_{center}_{system}_{v}_{ti}.nc")
print(pdict_list[0])
print(nm_list[0])

@ct.application(title='Get seasonal forecast daily data on single levels')
@ct.output.download()
def application():
    
    #ref_grid = ct.catalogue.retrieve(
    #    "reanalysis-era5-land",\
    #     {"variable" : "2m_temperature","time" : '12:00',"day" : '15',"month" : '08', "year" : '2000', 'area': bbox, 'grid': grid})

    def get_data(pdict, nm):
      ds = ct.catalogue.retrieve('seasonal-original-single-levels', pdict)
      dss = [ct.cube.box_select(d,lat=[bbox[0],bbox[2]], lon=[bbox[1],bbox[3]]) for d in ds]
      attrs = {'model': f"{pdict.get('originating_centre')}_{pdict.get('system')}", 'file_name':nm}
      dsss = [ct.cdm.update_attributes(d, attrs) for d in dss]
      #dssss = [ct.geo.regrid(d, reference_grid_data = ref_grid) for d in dsss]
      return dsss
    
    return [get_data(pdict, nm) for pdict, nm in zip(pdict_list, nm_list)]



Overwriting download.py


In [22]:
%%time
# This can take quite some time on the first run (hours)
# Once the request is made this cell can be cancelled, and results are cached for future runs
!python download.py

2020-03-06 11:45:53,631 INFO Welcome to the CDS
2020-03-06 11:45:53,631 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/run_workflow/clientid-04d90b902a7e41db8bfcfd73eb0ef4ed
2020-03-06 11:45:54,328 INFO Request is completed
2020-03-06 11:45:54,328 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data7/a7154230-9fae-4f2a-8c77-9fb2f8322d0c.nc to a7154230-9fae-4f2a-8c77-9fb2f8322d0c.nc (9.4M)
2020-03-06 11:45:55,761 INFO Download rate 6.6M/s
2020-03-06 11:45:55,761 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data7/b56653c7-b8b9-4c73-96a8-a0e8730c0a9c.nc to b56653c7-b8b9-4c73-96a8-a0e8730c0a9c.nc (9.4M)
2020-03-06 11:45:57,016 INFO Download rate 7.5M/s
2020-03-06 11:45:57,016 INFO Downloading http://136.156.132.198/cache-compute-0003/cache/data7/61bb936e-0fda-4b4f-88be-5c0816558425.nc to 61bb936e-0fda-4b4f-88be-5c0816558425.nc (13.4M)
2020-03-06 11:45:58,539 INFO Download rate 8.8M/s
2020-03-06 

In [0]:
# View example
#import xarray as xr
#import glob
#fl = glob.glob("*.nc")
#xr.set_options(display_style='html')
#ds = xr.open_dataset(fl[1])
#ds.tasmin

### Rescale and rename netcdfs

In [24]:
import xarray as xr
import glob
import os
import pprint
import datetime
xr.set_options(display_style='html')
import pandas as pd 

def match_grid(ds_target, ds_source):
  return ds_target.interp(lat=ds_source['lat'], lon=ds_source['lon'])

def fix_time(ds, start, end, freq='D'):
  a ={'standard_name': 'time',
  'long_name':         'time',
  'bounds':            'time_bnds',
  'axis':              'T',
  'stored_direction':  'increasing',
  'type':              'double'}
  ds = ds.sel(time=slice(start, end))
  ds.coords['time'].attrs = a
  ds.coords['time'] = xr.cftime_range(start=start, periods=len(ds.coords['time']), freq=freq, calendar='noleap')
  return xr.decode_cf(ds)  

# Get file list
fl = glob.glob("*.nc")
print(fl)

# Set reference grid
ds_source = xr.open_dataset(fl[1])

# Fix and export to netcdf   
for f in fl:
  ds = xr.open_dataset(f)
  # make cftime
  ds = xr.decode_cf(ds)
  # interpolate grid
  ds = match_grid(ds, ds_source)
  # get metadata
  dattrs = ds.tasmin.attrs
  # fix time
  sd = pd.to_datetime(str(ds.time[0].values)).strftime('%Y-%m-%d')
  ed = pd.to_datetime(str(ds.time[-1].values)).strftime('%Y-%m-%d')
  ds = fix_time(ds, sd, ed)
  ti = f"{sd}--{ed}"
  # make model a coord
  m = dattrs['model'].replace("_", "-")
  print(m)
  ds.coords['model'] = m
  ds.set_coords(['lat', 'lon', 'time', 'model', 'realization'])
  # make dir
  os.mkdir(os.path.join(ds_dir, "seasonal-original-single-levels", m))
  # make name
  nm = f"daily_seasonal-original-single-levels_tasmin-tasmax_{m}_{ti}.nc"
  print(nm)
  # export to netcdf
  #print(ds.coords)
  ds.to_netcdf(os.path.join(ds_dir,"seasonal-original-single-levels", m, nm))

['b0e04338-bf11-4adb-bea3-dc55276f255b.nc', 'b56653c7-b8b9-4c73-96a8-a0e8730c0a9c.nc', '8e0cd612-0009-4dfe-ad33-a86ced386797.nc', '61bb936e-0fda-4b4f-88be-5c0816558425.nc', 'a7154230-9fae-4f2a-8c77-9fb2f8322d0c.nc', '337471a0-5ea8-483d-bb5a-4b8aa74d9dcf.nc']
meteo-france-7
daily_seasonal-original-single-levels_tasmin-tasmax_meteo-france-7_2019-10-02--2020-04-30.nc
dwd-2
daily_seasonal-original-single-levels_tasmin-tasmax_dwd-2_2019-10-02--2020-03-31.nc
ukmo-14
daily_seasonal-original-single-levels_tasmin-tasmax_ukmo-14_2019-10-02--2020-05-03.nc
ecmwf-5
daily_seasonal-original-single-levels_tasmin-tasmax_ecmwf-5_2019-10-02--2020-05-03.nc
cmcc-3
daily_seasonal-original-single-levels_tasmin-tasmax_cmcc-3_2019-10-02--2020-03-31.nc
ncep-2
daily_seasonal-original-single-levels_tasmin-tasmax_ncep-2_2019-10-02--2020-05-03.nc


### Copy netcdfs to GCS

In [25]:
import os
copy_gcs([os.path.join(ds_dir, "seasonal-original-single-levels", "*")], [os.path.join(gs_prefix, ds_dir, "seasonal-original-single-levels")])

Processing: gsutil -m cp -r  dataset/seasonal-original-single-levels/* gs://copernicus-climate/dataset/seasonal-original-single-levels
Task created
Finished copy


### Open consolidated netcdf dataset

In [26]:
import os
copy_gcs([os.path.join(gs_prefix, ds_dir, "seasonal-original-single-levels")], [os.path.join(ds_dir)])

Processing: gsutil -m cp -r  gs://copernicus-climate/dataset/seasonal-original-single-levels dataset
Task created
Finished copy


In [27]:
import os
import glob

models = os.listdir(f"{ds_dir}/seasonal-original-single-levels")
print(models)
fl = [glob.glob(f"{ds_dir}/seasonal-original-single-levels/{m}/*.nc") for m in models]
print(fl)
ds = xr.open_mfdataset(fl, combine='nested', concat_dim=['model', None])
ds

['dwd-2', 'ukmo-14', 'ncep-2', 'meteo-france-7', 'cmcc-3', 'ecmwf-5']
[['dataset/seasonal-original-single-levels/dwd-2/daily_seasonal-original-single-levels_tasmin-tasmax_dwd-2_2019-10-02--2020-03-31.nc'], ['dataset/seasonal-original-single-levels/ukmo-14/daily_seasonal-original-single-levels_tasmin-tasmax_ukmo-14_2019-10-02--2020-05-03.nc'], ['dataset/seasonal-original-single-levels/ncep-2/daily_seasonal-original-single-levels_tasmin-tasmax_ncep-2_2019-10-02--2020-05-03.nc'], ['dataset/seasonal-original-single-levels/meteo-france-7/daily_seasonal-original-single-levels_tasmin-tasmax_meteo-france-7_2019-10-02--2020-04-30.nc'], ['dataset/seasonal-original-single-levels/cmcc-3/daily_seasonal-original-single-levels_tasmin-tasmax_cmcc-3_2019-10-02--2020-03-31.nc'], ['dataset/seasonal-original-single-levels/ecmwf-5/daily_seasonal-original-single-levels_tasmin-tasmax_ecmwf-5_2019-10-02--2020-05-03.nc']]


,Array,Chunk
Bytes,71.05 MB,11.84 MB
Shape,"(6, 1, 51, 215, 9, 15)","(1, 1, 51, 215, 9, 15)"
Count,73 Tasks,6 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,71.05 MB,11.84 MB
Shape,"(6, 1, 51, 215, 9, 15)","(1, 1, 51, 215, 9, 15)"
Count,72 Tasks,6 Chunks
Type,float64,numpy.ndarray


### Create ZARR

In [28]:
import zarr
# Create zarr store structure
store = ds.to_zarr(store=zarr_dir, group="seasonal-original-single-levels", mode="w", consolidated=True)
unchunk_dataset(dataset_path=f"{zarr_dir}/seasonal-original-single-levels", coords_only = True)
zarr.consolidate_metadata(zarr_dir)
root = zarr.open(zarr_dir, mode='r')
print(root.tree())

/
 └── seasonal-original-single-levels
     ├── forecast_reference_time (1,) int64
     ├── lat (9,) float64
     ├── lon (15,) float64
     ├── model (6,) object
     ├── realization (51,) int64
     ├── tasmax (6, 1, 51, 215, 9, 15) float64
     ├── tasmin (6, 1, 51, 215, 9, 15) float64
     └── time (215,) int64


In [29]:
# Check local ZARR
import xarray as xr
ds = xr.open_zarr(zarr_dir, group="seasonal-original-single-levels", consolidated=True)
ds

,Array,Chunk
Bytes,71.05 MB,11.84 MB
Shape,"(6, 1, 51, 215, 9, 15)","(1, 1, 51, 215, 9, 15)"
Count,7 Tasks,6 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,71.05 MB,11.84 MB
Shape,"(6, 1, 51, 215, 9, 15)","(1, 1, 51, 215, 9, 15)"
Count,7 Tasks,6 Chunks
Type,float64,numpy.ndarray


### Copy ZARR to GCS

In [30]:
import os
copy_gcs([zarr_dir], [gs_prefix])

Processing: gsutil -m cp -r  spain.zarr gs://copernicus-climate
Task created
Finished copy


In [31]:
# set bucket permissions to to public
set_acl_to_public(gs_prefix)

gsutil -m acl -r ch -u AllUsers:R gs://copernicus-climate
Set acl(s) sucsessful


### Check remote ZARR

In [32]:
ds = get_cached_zarr("seasonal-original-single-levels")
ds

,Array,Chunk
Bytes,71.05 MB,11.84 MB
Shape,"(6, 1, 51, 215, 9, 15)","(1, 1, 51, 215, 9, 15)"
Count,7 Tasks,6 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,71.05 MB,11.84 MB
Shape,"(6, 1, 51, 215, 9, 15)","(1, 1, 51, 215, 9, 15)"
Count,7 Tasks,6 Chunks
Type,float64,numpy.ndarray


### Clean up

In [0]:
# Clean up
import os
import glob
dummy = [os.remove(f) for f in glob.glob('*.nc')]

## CMIP5 daily data on single levels

### Get data from CDS toolbox

In [34]:
%%writefile download.py
import cdstoolbox as ct

# Out directory
ds_dir = "dataset"

# Define region of interest
bbox = [35, -10, 44, 5]
grid = [0.1, 0.1]

# Define datasets to download
exp_list = ['rcp_4_5', 'rcp_8_5']
var_list = ['minimum_2m_temperature_in_the_last_24_hours', 'maximum_2m_temperature_in_the_last_24_hours']
model_list = ['bnu_esm', 'access1_0', 'mpi_esm_lr', 'mpi_esm_mr', 'noresm1_m']
ensemble_list = ['r1i1p1'] #, 'r2i1p1', 'r3i1p1'
period_list = [
               ['20060101-21001231'],
               ['20060101-20301231', '20310101-20551231', '20560101-20801231','20810101-21001231'],
               ['20060101-20091231', '20100101-20191231', '20200101-20291231',
                '20300101-20391231', '20400101-20491231', '20500101-20591231',
                '20600101-20691231', '20700101-20791231', '20800101-20891231',
                '20900101-21001231'],
               ['20060101-20091231', '20100101-20191231', '20200101-20291231',
                '20300101-20391231', '20400101-20491231', '20500101-20591231',
                '20600101-20691231', '20700101-20791231', '20800101-20891231',
                '20900101-21001231'],
               ['20060101-20551231', '20560101-21001231']
               ]


# Function to create properties dict for download
def create_cmip5_dict(experiment, model, ensemble, variable, period):
  return {
        'ensemble_member': ensemble,
        'experiment': experiment,
        'variable': [variable],
        'model': model,
        'period': period
    }

# Make list of properties dicts, and out names
pdict_list = []
nm_list = []
for experiment in exp_list:
  for variable in var_list:
    for model, period in zip(model_list, period_list):
      for ensemble in ensemble_list:
        pdict_list.append(create_cmip5_dict(experiment, model, ensemble, variable, period))
        
        # make file names
        if len(period) >1:
          ti = f"{period[0].split('-')[0]}-{period[-1].split('-')[1]}"
        else:
          ti = period[0] 
        e = experiment.replace("_", "-")
        m = model.replace("_", "-")
        v = variable.replace("_", "-")
        en = ensemble.replace("_", "-")
        nm_list.append(f"{ds_dir}/bbox-{bbox[0]}-{bbox[1]}-{bbox[2]}-{bbox[3]}_{e}_{m}_{en}_{v}_{ti}.nc")
print(pdict_list[0])

@ct.application(title='Get CMIP5')
@ct.output.download()
def application():
    
    #ref_grid = ct.catalogue.retrieve(
    #    "reanalysis-era5-land",\
    #     {"variable" : "2m_temperature","time" : '12:00',"day" : '15',"month" : '08', "year" : '2000', 'area': bbox, 'grid': grid})

    def get_data(pdict, nm):
      ds = ct.catalogue.retrieve('projections-cmip5-daily-single-levels', pdict)
      dss = ct.cube.box_select(ds,lat=[bbox[0],bbox[2]], lon=[bbox[1],bbox[3]])
      dsss = ct.cdm.update_attributes(dss, pdict)
      #dssss = ct.geo.regrid(dsss, reference_grid_data = ref_grid)
      return dsss
    
    return [get_data(pdict, nm) for pdict, nm in zip(pdict_list, nm_list)]



Overwriting download.py


In [35]:
%%time
# This can take quite some time on the first run (hours)
# Once the request is made this cell can be cancelled, and results are cached for future runs
!python download.py

2020-03-06 11:46:24,938 INFO Welcome to the CDS
2020-03-06 11:46:24,939 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/run_workflow/clientid-1564d311f9594604aa1dd32caaffd89b
2020-03-06 11:46:25,545 INFO Request is completed
2020-03-06 11:46:25,545 INFO Downloading http://136.156.132.236/cache-compute-0007/cache/data0/cdm.update_attributes-1583483649.0597036-32753-35-6c4ff8d6-e20e-4a76-bfd6-3f61f177b975updated_attrs.nc to cdm.update_attributes-1583483649.0597036-32753-35-6c4ff8d6-e20e-4a76-bfd6-3f61f177b975updated_attrs.nc (2.3M)
2020-03-06 11:46:26,534 INFO Download rate 2.3M/s
2020-03-06 11:46:26,534 INFO Downloading http://136.156.132.105/cache-compute-0000/cache/data1/cdm.update_attributes-1583407227.5471609-14837-13-22c18d28-ec9d-48d5-bb4b-968b9cd36d56updated_attrs.nc to cdm.update_attributes-1583407227.5471609-14837-13-22c18d28-ec9d-48d5-bb4b-968b9cd36d56updated_attrs.nc (8.8M)
2020-03-06 11:46:27,943 INFO Download rate 6.

### Rename netcdfs

In [36]:
import xarray as xr
import glob
import os
import pprint
xr.set_options(display_style='html')

fl = glob.glob("*.nc")
print(fl)
for f in fl:
  ds = xr.open_dataset(f)
  ds = xr.decode_cf(ds)
  attrs = ds.attrs
  #pprint.pprint(attrs)
  if attrs['comment'].split(" ")[0] == 'maximum':
    v = 'tasmax'
  if attrs['comment'].split(" ")[0] == 'minimum':
    v = 'tasmin'
  dvattr = ds[v].attrs
  #pprint.pprint(dvattr)
  try:
    m = dvattr['model_id']
    e = dvattr['experiment_id']
    r = dvattr['parent_experiment_rip']
    nm = f"daily_projections-cmip5-daily-single-levels_{v}_{m}_{e}_{r}.nc"
  except:
    txt = dvattr['associated_files'].split('areacella:')[1]
    m = txt.split("_")[2]
    e = txt.split("_")[3]
    r = txt.split("_")[4]
    nm = f"daily_projections-cmip5-daily-single-levels_{v}_{m}_{e}_{r}"

  print(nm)
  ds.to_netcdf(nm)

['cdm.update_attributes-1583408337.980602-3350-29-5072415b-d02f-41df-9205-9e7e85f7ccb0updated_attrs.nc', 'cdm.update_attributes-1583485088.2598054-6649-34-4ba7fe67-b116-45e8-9484-26711d5a7a0bupdated_attrs.nc', 'cdm.update_attributes-1583484650.91409-24360-32-c8f78681-4397-4e05-bf3c-ba7f308d898bupdated_attrs.nc', 'cdm.update_attributes-1583330018.091668-18934-31-72f0280c-ae4d-4882-8291-590337ea7ba2updated_attrs.nc', 'cdm.update_attributes-1583332907.2565362-8613-17-16e61f06-77c8-4c7d-ab3f-b460593f48c6updated_attrs.nc', 'cdm.update_attributes-1583483649.0597036-32753-35-6c4ff8d6-e20e-4a76-bfd6-3f61f177b975updated_attrs.nc', 'cdm.update_attributes-1583334638.0098908-24574-37-f864a7f1-5bd0-4884-b477-3bc683ea6867updated_attrs.nc', 'cdm.update_attributes-1583407227.5471609-14837-13-22c18d28-ec9d-48d5-bb4b-968b9cd36d56updated_attrs.nc', 'cdm.update_attributes-1583335013.1150239-4249-29-d5833e02-07e9-4707-a590-7f6a68a7ef09updated_attrs.nc', 'cdm.update_attributes-1583333781.3958895-6439-19-bdb

### Merge, rescale, fix issues, and rename netcdfs

In [37]:
import xarray as xr
import numpy as np
import glob
import os

def concat_dicts(*dicts):
    keys = set().union(*dicts)
    #print(keys)
    return { k: ", ".join(list(set(dic.get(k) for dic in dicts)))  for k in keys}

def merge_vars(m, s):
  with xr.open_dataset(glob.glob(f"daily_projections-cmip5-daily-single-levels_tasmin_{m}_{s}_*.nc")[0])as ds1:
    ds1.coords['experiment'] = s
    ds1.coords['model'] = m
    ds1.coords['realization'] = np.array([1])
  
  with xr.open_dataset(glob.glob(f"daily_projections-cmip5-daily-single-levels_tasmax_{m}_{s}_*.nc")[0]) as ds2:
    ds2.coords['experiment'] = s
    ds2.coords['model'] = m
    ds2.coords['realization'] = np.array([1])

  out = xr.merge([ds1, ds2])
  out.set_coords(['lat', 'lon', 'time', 'experiment', 'model', 'realization'])
  out.attrs = concat_dicts(ds1.attrs, ds2.attrs)
  out.attrs['source'] = ", ".join([out.attrs['source'], 'CDS']) 
  return out

def match_grid(ds_target, ds_source):
  return ds_target.interp(lat=ds_source['lat'], lon=ds_source['lon'])

def fix_time(ds):
  a ={'standard_name': 'time',
  'long_name':         'time',
  'bounds':            'time_bnds',
  'axis':              'T',
  'stored_direction':  'increasing',
  'type':              'double'}
  ds = ds.sel(time=slice('2006-01-01', '2099-12-31'))
  t = ds.coords['time'].attrs = a
  t = ds.coords['time']
  ds.coords['time'] = xr.cftime_range(start='2006', periods=len(t.values), freq='D', calendar='noleap')
  return xr.decode_cf(ds)  


fl = glob.glob('daily_projections-cmip5-daily-single-levels_*.nc')
print(fl)
models = list(set([f.split("_")[3] for f in fl]))
variables = list(set([f.split("_")[2] for f in fl]))
scenarios = list(set([f.split("_")[4] for f in fl]))
print(models)
print(variables)
print(scenarios)

ds_source = xr.open_dataset('daily_projections-cmip5-daily-single-levels_tasmin_ACCESS1-0_rcp45_r0i0p0.nc')

for s in scenarios:
  print(s)
  os.mkdir(os.path.join(ds_dir, 'projections-cmip5-daily-single-levels', s))
  for m in models:
    print(m)
    os.mkdir(os.path.join(ds_dir, 'projections-cmip5-daily-single-levels', s, m))
    out = merge_vars(m,s)
    out = match_grid(out, ds_source)
    out = fix_time(out)
    #print(out.time) 
    out.to_netcdf(os.path.join(ds_dir, 'projections-cmip5-daily-single-levels', s, m, f"daily_projections-cmip5-daily-single-levels_tasmin-tasmax_{m}_{s}_r1i1p1.nc"))

ds_source.close()    

['daily_projections-cmip5-daily-single-levels_tasmax_NorESM1-M_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_ACCESS1-0_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_MPI-ESM-LR_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_BNU-ESM_rcp45_r1i1p1.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_BNU-ESM_rcp85_r1i1p1.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_ACCESS1-0_rcp45_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_NorESM1-M_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_NorESM1-M_rcp45_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_MPI-ESM-LR_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_ACCESS1-0_rcp45_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmin_MPI-ESM-MR_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-levels_tasmax_MPI-ESM-MR_rcp85_r0i0p0.nc', 'daily_projections-cmip5-daily-single-l

### Copy netcdfs to GCS

In [38]:
import os
copy_gcs([os.path.join(ds_dir, "projections-cmip5-daily-single-levels", "*")], [os.path.join(gs_prefix, ds_dir, "projections-cmip5-daily-single-levels")])

Processing: gsutil -m cp -r  dataset/projections-cmip5-daily-single-levels/* gs://copernicus-climate/dataset/projections-cmip5-daily-single-levels
Task created
Finished copy


### Open consolidated netcdf dataset

In [39]:
import os
copy_gcs([os.path.join(gs_prefix, ds_dir, "projections-cmip5-daily-single-levels")], [os.path.join(ds_dir)])

Processing: gsutil -m cp -r  gs://copernicus-climate/dataset/projections-cmip5-daily-single-levels dataset
Task created
Finished copy


In [40]:
import os
import glob
scenarios = os.listdir(os.path.join(ds_dir, "projections-cmip5-daily-single-levels"))
print(scenarios)
models = [os.listdir(os.path.join(ds_dir, "projections-cmip5-daily-single-levels", s)) for s in scenarios][0]
print(models)
fl = [[glob.glob(f"{ds_dir}/projections-cmip5-daily-single-levels/{s}/{m}/*.nc") for m in models] for s in scenarios]
print(fl)
ds = xr.open_mfdataset(fl, combine='nested', concat_dim=['experiment', 'model', 'realization'])\
.sel(time=slice('2006-01-01', '2099-12-31'))
ds

['rcp85', 'rcp45']
['BNU-ESM', 'MPI-ESM-MR', 'ACCESS1-0', 'NorESM1-M', 'MPI-ESM-LR']
[[['dataset/projections-cmip5-daily-single-levels/rcp85/BNU-ESM/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_BNU-ESM_rcp85_r1i1p1.nc'], ['dataset/projections-cmip5-daily-single-levels/rcp85/MPI-ESM-MR/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_MPI-ESM-MR_rcp85_r1i1p1.nc'], ['dataset/projections-cmip5-daily-single-levels/rcp85/ACCESS1-0/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_ACCESS1-0_rcp85_r1i1p1.nc'], ['dataset/projections-cmip5-daily-single-levels/rcp85/NorESM1-M/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_NorESM1-M_rcp85_r1i1p1.nc'], ['dataset/projections-cmip5-daily-single-levels/rcp85/MPI-ESM-LR/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_MPI-ESM-LR_rcp85_r1i1p1.nc']], [['dataset/projections-cmip5-daily-single-levels/rcp45/BNU-ESM/daily_projections-cmip5-daily-single-levels_tasmin-tasmax_BNU-ESM_rcp45_r1i1p1.nc'], ['dataset

,Array,Chunk
Bytes,175.67 MB,17.57 MB
Shape,"(5, 2, 34310, 8, 8, 1)","(1, 1, 34310, 8, 8, 1)"
Count,128 Tasks,10 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,175.67 MB,17.57 MB
Shape,"(5, 2, 34310, 8, 8, 1)","(1, 1, 34310, 8, 8, 1)"
Count,128 Tasks,10 Chunks
Type,float64,numpy.ndarray


### Create ZARR

In [41]:
import zarr
# Create zarr store structure
store = ds.to_zarr(store=zarr_dir, group="projections-cmip5-daily-single-levels", mode="w", consolidated=True)
unchunk_dataset(dataset_path=f"{zarr_dir}/projections-cmip5-daily-single-levels", coords_only = True)
zarr.consolidate_metadata(zarr_dir)
root = zarr.open(zarr_dir, mode='r')
print(root.tree())

/
 ├── projections-cmip5-daily-single-levels
 │   ├── experiment (2,) object
 │   ├── height (5,) float64
 │   ├── lat (8,) float64
 │   ├── lon (8,) float64
 │   ├── model (5,) object
 │   ├── realization (1,) int64
 │   ├── tasmax (5, 2, 34310, 8, 8, 1) float64
 │   ├── tasmin (5, 2, 34310, 8, 8, 1) float64
 │   └── time (34310,) int64
 └── seasonal-original-single-levels
     ├── forecast_reference_time (1,) int64
     ├── lat (9,) float64
     ├── lon (15,) float64
     ├── model (6,) object
     ├── realization (51,) int64
     ├── tasmax (6, 1, 51, 215, 9, 15) float64
     ├── tasmin (6, 1, 51, 215, 9, 15) float64
     └── time (215,) int64


In [42]:
# Check local ZARR
import xarray as xr
ds = xr.open_zarr(zarr_dir, group="projections-cmip5-daily-single-levels", consolidated=True)
ds

<xarray.Dataset>
Dimensions:      (experiment: 2, lat: 8, lon: 8, model: 5, realization: 1, time: 34310)
Coordinates:
  * experiment   (experiment) object 'rcp85' 'rcp45'
    height       (model) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * lat          (lat) float64 35.0 36.25 37.5 38.75 40.0 41.25 42.5 43.75
  * lon          (lon) float64 -9.375 -7.5 -5.625 -3.75 -1.875 0.0 1.875 3.75
  * model        (model) object 'BNU-ESM' 'MPI-ESM-MR' ... 'MPI-ESM-LR'
  * realization  (realization) int64 1
  * time         (time) object 2006-01-01 00:00:00 ... 2099-12-31 00:00:00
Data variables:
    tasmax       (model, experiment, time, lat, lon, realization) float64 dask.array<chunksize=(1, 1, 34310, 8, 8, 1), meta=np.ndarray>
    tasmin       (model, experiment, time, lat, lon, realization) float64 dask.array<chunksize=(1, 1, 34310, 8, 8, 1), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.4
    comment:        minimum near-surface (usually, 2 meter) air temperature (...
    contact:        Ji Duoying (duoyingji@bnu.edu.cn)
    creation_date:  2012-04-11T10:34:34Z
    history:        2012-04-11T10:34:29Z CMOR rewrote data to comply with CF ...
    institution:    GCESS,BNU,Beijing,China
    source:         BNU-ESM, CDS
    title:          BNU-ESM model output prepared for CMIP5 RCP8.5

### Copy ZARR to GCS

In [43]:
import os
copy_gcs([zarr_dir], [gs_prefix])

Processing: gsutil -m cp -r  spain.zarr gs://copernicus-climate
Task created
Finished copy


In [44]:
# set bucket permissions to to public
set_acl_to_public(gs_prefix)

gsutil -m acl -r ch -u AllUsers:R gs://copernicus-climate
Set acl(s) sucsessful


### Check remote ZARR

In [45]:
ds = get_cached_zarr("projections-cmip5-daily-single-levels")
ds

<xarray.Dataset>
Dimensions:      (experiment: 2, lat: 8, lon: 8, model: 5, realization: 1, time: 34310)
Coordinates:
  * experiment   (experiment) object 'rcp85' 'rcp45'
    height       (model) float64 dask.array<chunksize=(5,), meta=np.ndarray>
  * lat          (lat) float64 35.0 36.25 37.5 38.75 40.0 41.25 42.5 43.75
  * lon          (lon) float64 -9.375 -7.5 -5.625 -3.75 -1.875 0.0 1.875 3.75
  * model        (model) object 'BNU-ESM' 'MPI-ESM-MR' ... 'MPI-ESM-LR'
  * realization  (realization) int64 1
  * time         (time) object 2006-01-01 00:00:00 ... 2099-12-31 00:00:00
Data variables:
    tasmax       (model, experiment, time, lat, lon, realization) float64 dask.array<chunksize=(1, 1, 34310, 8, 8, 1), meta=np.ndarray>
    tasmin       (model, experiment, time, lat, lon, realization) float64 dask.array<chunksize=(1, 1, 34310, 8, 8, 1), meta=np.ndarray>
Attributes:
    Conventions:    CF-1.4
    comment:        minimum near-surface (usually, 2 meter) air temperature (...
    contact:        Ji Duoying (duoyingji@bnu.edu.cn)
    creation_date:  2012-04-11T10:34:34Z
    history:        2012-04-11T10:34:29Z CMOR rewrote data to comply with CF ...
    institution:    GCESS,BNU,Beijing,China
    source:         BNU-ESM, CDS
    title:          BNU-ESM model output prepared for CMIP5 RCP8.5

### Clean up

In [0]:
# Clean up
import os
import glob
dummy = [os.remove(f) for f in glob.glob('*.nc')]

## ERA5-Land-hourly-data-from-1981-to-present

### Get data from CDS toolbox

In [47]:
%%writefile download.py
import cdstoolbox as ct

# Out directory
ds_dir = "dataset"

# Define region of interest
bbox = [35, -10, 44, 5]
grid = [0.1, 0.1]
# Define datasets to download
# GOTYA remember range needs to be +1
var_list = ["2m_temperature"]
times = [f"0{t}:00" if t < 10 else f"{t}:00"  for t in range(0, 23+1)]
days = [f"0{t}" if t < 10 else f"{t}"  for t in range(1, 31+1)]
months = [f"0{t}" if t < 10 else f"{t}"  for t in range(1, 12+1)]
years = [
         [str(t) for t in range(1980, 1989+1)],
         [str(t) for t in range(1990, 1999+1)],
         [str(t) for t in range(2000, 2009+1)],
         [str(t) for t in range(2010, 2019+1)]
         ]

# Function to create properties dict for download
def create_prop_dict(variable, days, months, years, bbox, grid):
  return {
      "variable" : variable,
      "time" : times,
      "day" : days,
      "month" : months,
      "year" : years,
      'area': bbox,
      'grid': grid
      }

# Make list of properties dicts, and out names
pdict_list = []
nm_list = []
for variable in var_list:
  for year in years:
      pdict_list.append(create_prop_dict(variable, days, months, year, bbox, grid))
      # make file names
      if len(years) >1:
        ti = f"{year[0]}-{year[-1]}"
      else:
        ti = year[0] 
      v = variable.replace("_", "-")
      nm_list.append(f"bbox-{bbox[0]}-{bbox[1]}-{bbox[2]}-{bbox[3]}_{v}_{ti}.nc")

@ct.output.download()
def application():
    
    def get_data(pdict, nm):
      ds = ct.catalogue.retrieve('reanalysis-era5-land', pdict)
      data_daily_max = ct.cdm.rename(ct.climate.daily_max(ds),"tasmax")
      data_daily_min = ct.cdm.rename(ct.climate.daily_min(ds),"tasmin")
      return [data_daily_max, data_daily_min]
    
    return [get_data(pdict, nm) for pdict, nm in zip(pdict_list, nm_list)]

Overwriting download.py


In [48]:
%%time
# This can take quite some time on the first run (hours)
# Once the request is made this cell can be cancelled, and results are cached for future runs
!python download.py

2020-03-06 11:49:53,951 INFO Welcome to the CDS
2020-03-06 11:49:53,952 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/tasks/services/tool/toolbox/orchestrator/run_workflow/clientid-210f448b10a649aa96337741e1adf564
2020-03-06 11:49:54,848 INFO Request is completed
2020-03-06 11:49:54,848 INFO Downloading http://136.156.133.37/cache-compute-0011/cache/data9/82a3eca7-b5f8-45cf-ad80-5d313d14a512.nc to 82a3eca7-b5f8-45cf-ad80-5d313d14a512.nc (344.6M)
2020-03-06 11:50:22,962 INFO Download rate 12.3M/s
2020-03-06 11:50:22,962 INFO Downloading http://136.156.133.37/cache-compute-0011/cache/data9/901ac628-cbba-444e-bf08-ed8080c9391f.nc to 901ac628-cbba-444e-bf08-ed8080c9391f.nc (382.9M)
2020-03-06 11:51:39,456 INFO Download rate 5M/s
2020-03-06 11:51:39,457 INFO Downloading http://136.156.133.37/cache-compute-0011/cache/data9/6a75762f-da13-406d-9388-6286e712761a.nc to 6a75762f-da13-406d-9388-6286e712761a.nc (383M)
2020-03-06 11:52:35,283 INFO Download rate 6.9M/s
2020-03-06 1

In [49]:
import xarray as xr
ds = xr.open_dataset('/content/012cf27d-f8e1-441e-b863-d32fc9df6490.nc')
ds.tasmin

<xarray.DataArray 'tasmin' (realization: 1, time: 3652, lat: 91, lon: 151)>
[50182132 values with dtype=float32]
Coordinates:
  * time         (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2019-12-31
  * realization  (realization) int64 0
  * lat          (lat) float64 35.0 35.1 35.2 35.3 35.4 ... 43.7 43.8 43.9 44.0
  * lon          (lon) float64 -10.0 -9.9 -9.8 -9.7 -9.6 ... 4.6 4.7 4.8 4.9 5.0
Attributes:
    long_name:      Near-Surface Air Temperature
    units:          K
    standard_name:  air_temperature
    type:           real
    Conventions:    CF-1.7
    source:         ECMWF
    institution:    European Centre for Medium-Range Weather Forecasts
    comment:        near-surface (usually, 2 meter) air temperature
    history:        2020-03-06T02:08:05 GRIB to CDM+CF via cfgrib-0.9.7.7/ecC...

### Rename netcdfs

In [50]:
import xarray as xr
import glob
import os
import pprint
import datetime

def fix_time(ds, start, end, freq='D'):
  a ={'standard_name': 'time',
  'long_name':         'time',
  'bounds':            'time_bnds',
  'axis':              'T',
  'stored_direction':  'increasing',
  'type':              'double'}
  ds = ds.sel(time=slice('1981-01-01', '2020-01-01'))
  ds.coords['time'].attrs = a
  ds.coords['time'] = xr.cftime_range(start=start, periods=len(ds.coords['time']), freq=freq, calendar='noleap')
  return xr.decode_cf(ds)


fl = glob.glob("*.nc")
print(fl)
for f in fl:
  ds = xr.open_dataset(f)
  # make cftime
  ds = xr.decode_cf(ds)
  # fix time
  sd = pd.to_datetime(str(ds.time[0].values)).strftime('%Y-%m-%d')
  ed = pd.to_datetime(str(ds.time[-1].values)).strftime('%Y-%m-%d')
  #ds = fix_time(ds, sd, ed)
  ti = f"{sd}--{ed}"
  # set coords
  ds.set_coords(['lat', 'lon', 'time'])
  ds = ds.squeeze(drop='realization')
  # make name
  nm = f"daily_reanalysis-era5-land_tasmin-tasmax_{ti}.nc"
  print(nm)
  print(ds.time)
  # export to netcdf
  ds.to_netcdf(os.path.join(ds_dir,"reanalysis-era5-land", nm))

['82a3eca7-b5f8-45cf-ad80-5d313d14a512.nc', '901ac628-cbba-444e-bf08-ed8080c9391f.nc', '6a75762f-da13-406d-9388-6286e712761a.nc', '012cf27d-f8e1-441e-b863-d32fc9df6490.nc']
daily_reanalysis-era5-land_tasmin-tasmax_1981-01-01--1989-12-31.nc
<xarray.DataArray 'time' (time: 3287)>
array(['1981-01-01T00:00:00.000000000', '1981-01-02T00:00:00.000000000',
       '1981-01-03T00:00:00.000000000', ..., '1989-12-29T00:00:00.000000000',
       '1989-12-30T00:00:00.000000000', '1989-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 ... 1989-12-31
Attributes:
    long_name:         time
    standard_name:     time
    axis:              T
    stored_direction:  increasing
    type:              double
daily_reanalysis-era5-land_tasmin-tasmax_1990-01-01--1999-12-31.nc
<xarray.DataArray 'time' (time: 3652)>
array(['1990-01-01T00:00:00.000000000', '1990-01-02T00:00:00.000000000',
       '1990-01-03T00:00:00.000000000', ..., 

### Copy netcdfs to GCS

In [51]:
import os
copy_gcs([os.path.join(ds_dir, "reanalysis-era5-land", "*")], [os.path.join(gs_prefix, ds_dir, "reanalysis-era5-land")])

Processing: gsutil -m cp -r  dataset/reanalysis-era5-land/* gs://copernicus-climate/dataset/reanalysis-era5-land
Task created
Finished copy


### Open consolidated netcdf dataset

In [52]:
import os
copy_gcs([os.path.join(gs_prefix, ds_dir, "reanalysis-era5-land")], [ds_dir])
#!gsutil -m cp -r gs://copernicus-climate/dataset/reanalysis-era5-land/* dataset

Processing: gsutil -m cp -r  gs://copernicus-climate/dataset/reanalysis-era5-land dataset
Task created
Finished copy


In [53]:
import os
ds = xr.open_mfdataset(os.path.join(ds_dir, 'reanalysis-era5-land', '*.nc')).chunk({'time':365})
#.sel(time=slice('1981-01-01', '2020-01-01'))
ds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  
/usr/local/lib/python3.6/dist-packages/xarray/backends/api.py:933: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in future, please use the new
`combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  from_openmfds=True,


,Array,Chunk
Bytes,782.91 MB,20.06 MB
Shape,"(14244, 91, 151)","(365, 91, 151)"
Count,99 Tasks,40 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,782.91 MB,20.06 MB
Shape,"(14244, 91, 151)","(365, 91, 151)"
Count,99 Tasks,40 Chunks
Type,float32,numpy.ndarray


### Create ZARR

In [54]:
import zarr
# Create zarr store structure
store = ds.to_zarr(store=zarr_dir, group="reanalysis-era5-land", mode="w", consolidated=True)
unchunk_dataset(dataset_path=f"{zarr_dir}/reanalysis-era5-land", coords_only = True)
zarr.consolidate_metadata(zarr_dir)
root = zarr.open(zarr_dir, mode='r')
print(root.tree())

/
 ├── projections-cmip5-daily-single-levels
 │   ├── experiment (2,) object
 │   ├── height (5,) float64
 │   ├── lat (8,) float64
 │   ├── lon (8,) float64
 │   ├── model (5,) object
 │   ├── realization (1,) int64
 │   ├── tasmax (5, 2, 34310, 8, 8, 1) float64
 │   ├── tasmin (5, 2, 34310, 8, 8, 1) float64
 │   └── time (34310,) int64
 ├── reanalysis-era5-land
 │   ├── lat (91,) float64
 │   ├── lon (151,) float64
 │   ├── tasmax (14244, 91, 151) float32
 │   ├── tasmin (14244, 91, 151) float32
 │   └── time (14244,) int64
 └── seasonal-original-single-levels
     ├── forecast_reference_time (1,) int64
     ├── lat (9,) float64
     ├── lon (15,) float64
     ├── model (6,) object
     ├── realization (51,) int64
     ├── tasmax (6, 1, 51, 215, 9, 15) float64
     ├── tasmin (6, 1, 51, 215, 9, 15) float64
     └── time (215,) int64


### Copy ZARR to GCS

In [55]:
import os
copy_gcs([zarr_dir], [gs_prefix])

Processing: gsutil -m cp -r  spain.zarr gs://copernicus-climate
Task created
Finished copy


In [56]:
# Set to public
set_acl_to_public(gs_prefix)

gsutil -m acl -r ch -u AllUsers:R gs://copernicus-climate
Set acl(s) sucsessful


### Check remote ZARR

In [57]:
ds = get_cached_zarr("reanalysis-era5-land")
ds

,Array,Chunk
Bytes,782.91 MB,20.06 MB
Shape,"(14244, 91, 151)","(365, 91, 151)"
Count,41 Tasks,40 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,782.91 MB,20.06 MB
Shape,"(14244, 91, 151)","(365, 91, 151)"
Count,41 Tasks,40 Chunks
Type,float32,numpy.ndarray


### Clean up

In [0]:
# Clean up
import os
import glob
dummy = [os.remove(f) for f in glob.glob('*.nc')]

## Thermal-comfort-indices-derived-from-ERA5-reanalysis

In [0]:
#TODO